In [2]:
import sys
sys.path.insert(0, "./models")

from basic_vit import BasicViT

from stable_baselines3 import PPO
import gymnasium as gym
import gym
import time
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import gymnasium
from vizdoom import gymnasium_wrapper # This import will register all the environments
import os

from models.temporal_vit import TemporalViT
from models.infini_vit import InfiniViT

from ppo_basic import ViTFeatureExtractor


PyTorch device check: cpu


In [22]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create a **vectorized** VizDoom environment with 1 env
env = make_vec_env("VizdoomCorridor-v0", n_envs=1, env_kwargs={"render_mode": "human"})

# Load the trained PPO model
model = PPO.load(
    "ppo_basic_vizdoom",
    env=env,
    device=device,
    custom_objects={
        "features_extractor_class": ViTFeatureExtractor,
        "features_extractor_kwargs": dict(features_dim=512),
    }
)

# Run 3 episodes
for episode in range(1, 4):
    obs = env.reset()
    done = [False]
    total_reward = 0

    print(f"\n▶️ Starting episode {episode}")

    while not done[0]:  # VecEnv returns list-like done flags
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        total_reward += reward[0]
        time.sleep(0.03)

    print(f"✅ Episode {episode} finished with total reward: {total_reward}")

env.close()

Using device: cpu
Wrapping the env in a VecTransposeImage.
Observation space: Dict('gamevariables': Box(-3.4028235e+38, 3.4028235e+38, (1,), float32), 'screen': Box(0, 255, (3, 240, 320), uint8))
Shape from observation space: (3, 240, 320)
Extracted dimensions: h=84, w=84, c=3
Will create 36 patches (6x6) with patch_size=14
BasicViT model initialized on: cpu

▶️ Starting episode 1
✅ Episode 1 finished with total reward: 285.4735107421875

▶️ Starting episode 2
✅ Episode 2 finished with total reward: 306.26678466796875

▶️ Starting episode 3
✅ Episode 3 finished with total reward: 767.9105224609375
